In [18]:
%matplotlib inline
import numpy as np
from pulp import *
import pandas as pd
import pprint
import csv
MEPS=1.03-6

In [30]:
df=pd.read_csv('shop.csv',header=0,usecols=[0])
df

,店舗
0,Shop0
1,Shop1
2,Shop2
3,Shop3
4,Shop4
5,Shop5


In [31]:
DMUs=df.values
print(DMUs)

[['Shop0']
 ['Shop1']
 ['Shop2']
 ['Shop3']
 ['Shop4']
 ['Shop5']]


In [32]:
dfx=pd.read_csv('shop.csv',header=0,index_col=0,usecols=[0,1,2])
print(dfx)
print('')
datax=dfx.values
print(datax)

       店員数  稼働時間
店舗              
Shop0    5  24.0
Shop1   10  12.0
Shop2   20  12.0
Shop3   20  24.0
Shop4   30  12.2
Shop5   50  12.0

[[ 5.  24. ]
 [10.  12. ]
 [20.  12. ]
 [20.  24. ]
 [30.  12.2]
 [50.  12. ]]


In [22]:
dfy=pd.read_csv('shop.csv',header=0,usecols=[3])
dfy

,売り上げ
0,2
1,6
2,10
3,12
4,12
5,20


In [24]:
n,m=x.shape
n,s=y.shape

In [25]:
res=[]
vs_=[]
us_=[]

In [26]:
for o in range(n):
    prob=LpProblem('DMU_'+str(o),LpMaximize)
    #　重み
    v=[LpVariable('v'+str(i),lowBound=0,
                  cat='Continuous') for i in range(m)]
    u=[LpVariable('u'+str(i),lowBound=0,
                  cat='Continuous') for i in range(s)]
    #　目的関数
    prob += lpDot(u,y.iloc[o,:])

    # 制約条件
    prob += lpDot(v,x.iloc[o,:])==1,'Normalize'+str(o)

    for j in range(n):
        prob += lpDot(u,y.iloc[j,:]) <= lpDot(v,x.iloc[j,:])

    prob.solve()
    vs=np.array([v[i].varValue for i in range(m)]) #v*
    vs_.append([vs])
    us=np.array([u[i].varValue for i in range(s)]) #u*
    us_.append([us])
    print('DMU_'+str(o),': ',DMUs[o])
    print('vs_'+str(o),': ',vs)
    print('us_'+str(o),': ',us)
    print(' ')

    res.append([DMUs[o],value(prob.objective)])

DMU_0 :  ['Shop0']
vs_0 :  [0.2 0. ]
us_0 :  [0.33333333]
 
DMU_1 :  ['Shop1']
vs_1 :  [0.1 0. ]
us_1 :  [0.16666667]
 
DMU_2 :  ['Shop2']
vs_2 :  [0.04       0.01666667]
us_2 :  [0.1]
 
DMU_3 :  ['Shop3']
vs_3 :  [0.05 0.  ]
us_3 :  [0.08333333]
 
DMU_4 :  ['Shop4']
vs_4 :  [0.02489627 0.02074689]
us_4 :  [0.0746888]
 
DMU_5 :  ['Shop5']
vs_5 :  [0.         0.08333333]
us_5 :  [0.05]
 


In [29]:
df_f=pd.DataFrame(res,columns=['店舗番号','効率値'])
df_s=df_f.sort_values('効率値',ascending=False)
df_s

,店舗番号,効率値
1,[Shop1],1.000000
2,[Shop2],1.000000
5,[Shop5],1.000000
3,[Shop3],1.000000
4,[Shop4],0.896266
0,[Shop0],0.666667
